In [4]:
# Cargar Complementos
import os
import openai
import requests
from langchain_openai import AzureOpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_openai import AzureChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders.merge import MergedDataLoader

openai.api_type = "azure"
openai.api_version = "2024-02-01"

os.environ["AZURE_OPENAI_ENDPOINT"] = "https://edgasancdemo.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "57a41271b75040089f77ac694022a12d"

In [5]:
loader_pdf_1 = PyPDFLoader("http://python.edgasanc.com/docs/ag_2024/01.pdf")
loader_pdf_2 = PyPDFLoader("http://python.edgasanc.com/docs/ag_2024/02.pdf")
loader_pdf_3 = PyPDFLoader("http://python.edgasanc.com/docs/ag_2024/03.pdf")
loader_pdf_4 = PyPDFLoader("http://python.edgasanc.com/docs/ag_2024/04.pdf")
loader_pdf_5 = PyPDFLoader("http://python.edgasanc.com/docs/ag_2024/05.pdf")
loader_pdf_6 = PyPDFLoader("http://python.edgasanc.com/docs/ag_2024/06.pdf")
loader_pdf_7 = PyPDFLoader("http://python.edgasanc.com/docs/ag_2024/07.pdf")

In [8]:
loader_all = MergedDataLoader(loaders=[loader_pdf_1, loader_pdf_2, loader_pdf_3, loader_pdf_4, loader_pdf_5, loader_pdf_6, loader_pdf_7])

In [11]:
docs_all = loader_all.load()

In [13]:
len(docs_all)

146

In [18]:
type(docs_all), len(docs_all), docs_all[6]

(list,
 146,
 Document(page_content=' \nDepartamento Administrativo del Servicio Civil Distrital  \nCarrera 30 No 25 – 90, \nPiso 9 Costado Oriental.  \nTel: (57) 60 1 3680038  \nCódigo Postal: 111311  \nwww.serviciocivil.gov.co  \n  \n  \n \nPágina 7 de 9 - A-GDO -FM-008  Versión 8.0  24/01/2022  \n \n(Decreto 1227 de 2005, art.  100) \nARTÍCULO  2.2.13.1.4  Capacitación y desarrollo de los gerentes públicos . Las acciones de \ncapacitación y desarrollo de los gerentes públicos deberán establecerse a partir de la \nevaluación de los resultados de su gestión y orientarse a la identificación, definición y \nfortalecimiento de las competencias gerenciales.  \n(Decreto 1227 de 2005, art.  101) \nARTÍCULO  2.2.13.1.5  Evaluación de la gestión gerencial . La evaluación de la gestión \ngerencial se realizará con base en los Acuerdos de Gestión, documentos escritos y \nfirmados entre el superior jerárquico y el respectivo gerente público, con fundamento en \nlos planes, programas y proyectos 

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 1000,
    chunk_overlap= 200,
    length_function= len
    )

documents = text_splitter.split_documents(docs_all)

In [24]:
len(documents), documents[10]

(391,
 Document(page_content='Departamento Administrativo del Servicio Civil Distrital  \nCarrera 30 No 25 – 90, \nPiso 9 Costado Oriental.  \nTel: (57) 60 1 3680038  \nCódigo Postal: 111311  \nwww.serviciocivil.gov.co  \n  \n  \n \nPágina 5 de 9 - A-GDO -FM-008  Versión 8.0  24/01/2022  \n \n \n \n \nDECRETO 1083 DE 2015  \n(Mayo 26)  \nPor medio del cual se expide el Decreto Único Reglamentario del Sector de Función \nPública.  \n(…) \nCAPÍTULO 2  \nFUNCIONES DE LOS EMPLEOS SEGÚN EL NIVEL JERÁRQUICO  \nARTÍCULO  2.2.2.2.1  Nivel Directivo . Comprende los empleos a los cuales corresponden \nfunciones de dirección general, de formulación de políticas institucionales y de adopción \nde planes, programas y proyectos.  \nDe acuerdo con su naturaleza, los empleos de este nivel tendrán, entre otras, las \nsiguientes funciones:  \n1. Fijar las políticas y adoptar los planes generales relacionados con la institución o el \nsector al que pertenecen y velar por el cumplimiento de los términos y

In [27]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="ucs-embedding-large",
    openai_api_version="2024-02-01",
)

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings
)

In [30]:
base_datos = Chroma.from_documents(documents, embeddings, persist_directory="./chroma_db_ac_2024")
base_datos.persist()

/home/a8bc75c5-f115-4fea-81db-5cc5de88dd6c/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [34]:
retriever = base_datos.as_retriever(
    search_kwargs = {"k": 3}
)

In [37]:
from langchain.chains import RetrievalQA
from langchain_openai import AzureChatOpenAI

chat = AzureChatOpenAI(
    openai_api_version="2024-02-01",
    azure_deployment="ucs-chat-gpt",
    temperature=0.0
)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=retriever
)

In [40]:
query = "¿Qué son los acuerdos de gestion?"
qa_chain.run(query)

/home/a8bc75c5-f115-4fea-81db-5cc5de88dd6c/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Los Acuerdos de Gestión son instrumentos de evaluación gerencial mediante los cuales cada gerente público, junto con su superior jerárquico, formaliza los compromisos a desarrollar durante un determinado período o vigencia. Estos compromisos deben estar directamente relacionados con la visión, misión, objetivos, planes, programas o proyectos de la entidad u organismo y la dependencia a la cual pertenece el gerente público. La formalización de estos acuerdos incluye la estructuración y priorización de compromisos gerenciales y comportamentales a desarrollar y evaluar durante la respectiva vigencia.'

In [42]:
query = "¿Como fijo los acuerdos de gestión?"
qa_chain.run(query)

'Para fijar los Acuerdos de Gestión, debes seguir estos pasos:\n\n1. **Concertación de Compromisos**: El gerente público y su superior jerárquico deben estructurar y priorizar los compromisos gerenciales y comportamentales que se desarrollarán y evaluarán durante la vigencia del acuerdo. Estos compromisos deben estar alineados con la visión, misión, objetivos, planes, programas o proyectos de la entidad u organismo y la dependencia a la cual pertenece el gerente público.\n\n2. **Definición de Indicadores y Actividades**: Los compromisos deben ser medibles, realizables, verificables y cuantificables. Se deben establecer entre tres (3) y cinco (5) compromisos, incluyendo la definición de indicadores y la descripción de las actividades a través de las cuales se desarrollarán o ejecutarán los acuerdos establecidos. Cada compromiso tendrá un peso porcentual que oscilará entre un mínimo de 10% y un máximo de 40%, dependiendo de su relevancia y complejidad.\n\n3. **Formalización del Acuerdo**